In [2]:
import pandas as pd
import openpyxl
from itertools import product

def get_cell_color(cell):
    if cell.fill.start_color.index:
        return cell.fill.start_color.index
    return None

def get_values(value, color, column, style=None):
    if value == None:
        if column == 'CENTER STONE':
            if color == 'FF92D050':  # Green
                return ['Moissanite', 'Lab Diamond', 'Natural Diamond', 'Natural Champagne Diamond',
                        'Natural Cornflower Blue Sapphire', 'Natural Emerald', 'Natural Ruby']
            elif color == 'FFFFFF00':  # Yellow
                return ['Moissanite', 'Lab Diamond', 'Natural Diamond']
        elif column == 'BAND STYLE':
            if color == 'FFFFC000':  # Orange
                return ['Pave Diamond Sides', 'Plain Band']
        elif column == 'PROFILE':
            if color == 'FF00B0F0':  # Sky Blue
                return ['*Only Raised']
            elif color == 'FF7030A0':  # Purple
                return ['*Only Inline']
            return [value] if value else []
    return [value] if value else []

# Read the Excel file
file_path = 'Copy of Berlinger x Gemist - Product Sheet V1 -PARTNER.xlsx'
wb = openpyxl.load_workbook(file_path)
sheet = wb.active

# Get headers
headers = ['SETTING', 'STYLE', 'METAL', 'CENTER STONE', 'BAND STYLE', 'PROFILE']

# Read data and organize by SETTING
settings_data = {}
all_values = {header: set() for header in headers[1:]}

print("Reading data from Excel:")
current_setting = None
for row in sheet.iter_rows(min_row=2, max_col=len(headers), values_only=False):
    setting = row[0].value
    if setting:
        current_setting = setting
        if current_setting not in settings_data:
            settings_data[current_setting] = {header: set() for header in headers[1:]}
        print(f"New setting found: {current_setting}")
    
    if current_setting:
        for idx, cell in enumerate(row[1:], 1):
            value = cell.value
            color = get_cell_color(cell)
            # if value:
            values = get_values(value, color, headers[idx])
            settings_data[current_setting][headers[idx]].update(values)
            all_values[headers[idx]].update(values)
            print(f"  {headers[idx]}: {values}")

print("\nData collected:")
for setting, data in settings_data.items():
    print(f"Setting: {setting}")
    for header, values in data.items():
        print(f"  {header}: {values}")

# Generate combinations
all_combinations = []

print("\nGenerating combinations:")
for setting, data in settings_data.items():
    profiles = list(data['PROFILE'])
    if not profiles:
        profiles = all_values['PROFILE']

    if any(c == 'FF00B0F0' for c in profiles):  # If there's a sky blue color
        profiles = ['*Only Raised']
    elif any(c == 'FF7030A0' for c in profiles):  # If there's a purple color
        profiles = ['*Only Inline']
    else:
        profiles = list(set(profiles) | set(all_values['PROFILE']))

    setting_combinations = list(product(
        [setting],
        data['STYLE'] or all_values['STYLE'],
        data['METAL'] or all_values['METAL'],
        data['CENTER STONE'] or all_values['CENTER STONE'],
        data['BAND STYLE'] or all_values['BAND STYLE'],
        profiles
    ))
    all_combinations.extend(setting_combinations)
    print(f"Combinations for {setting}: {len(setting_combinations)}")

# Create DataFrame
combinations_df = pd.DataFrame(all_combinations, columns=headers)

# Remove rows where all columns except SETTING are empty
combinations_df = combinations_df.loc[combinations_df.iloc[:, 1:].any(axis=1)]

# Save to Excel
output_file_path = 'Combinations_Result_Final7.xlsx'
combinations_df.to_excel(output_file_path, index=False)

print(f"\nTotal combinations generated: {len(combinations_df)}")
print(f"Combinations saved to {output_file_path}")

Reading data from Excel:
New setting found: Oval
  STYLE: ['Oval Ballerina Round Center 1.00 ctw.']
  METAL: ['14K Yellow Gold']
  CENTER STONE: ['Moissanite']
  BAND STYLE: ['Antique Shape Diamond Sides']
  PROFILE: ['Inline']
  STYLE: ['Oval Ballerina Round Center 0.35 ctw']
  METAL: ['14K White Gold']
  CENTER STONE: ['Lab Diamond']
  BAND STYLE: ['Pave Diamond Sides ']
  PROFILE: ['Raised']
  STYLE: ['Oval Ballerina Oval Center']
  METAL: ['14K Rose Gold']
  CENTER STONE: ['Natural Diamond']
  BAND STYLE: ['Plain Band']
  PROFILE: []
  STYLE: ['Round with Feather Marquise']
  METAL: ['18K Yellow Gold']
  CENTER STONE: ['Natural Champagne Diamond']
  BAND STYLE: []
  PROFILE: []
  STYLE: ['Round with Marquise']
  METAL: ['18K White Gold']
  CENTER STONE: ['Natural Cornflower Blue Sapphire']
  BAND STYLE: []
  PROFILE: []
  STYLE: ['Sunflower']
  METAL: ['18K Rose Gold']
  CENTER STONE: ['Natural Emerald']
  BAND STYLE: []
  PROFILE: []
  STYLE: ['Flora']
  METAL: ['Platinum']
  CENT